<a href="https://colab.research.google.com/github/tc3oliver/LangChain-Guide/blob/dev/07_multiple_user_memory_local_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://s3-us-west-2.amazonaws.com/dynamodb-local/dynamodb_local_latest.tar.gz
!tar -zxvf dynamodb_local_latest.tar.gz

--2023-08-03 01:11:14--  https://s3-us-west-2.amazonaws.com/dynamodb-local/dynamodb_local_latest.tar.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.250.232, 52.92.153.224, 52.92.210.0, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.250.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44278951 (42M) [application/x-tar]
Saving to: ‘dynamodb_local_latest.tar.gz’

dynamodb_local_late 100%[===================>]  42.23M  14.2MB/s    in 3.0s    

2023-08-03 01:11:18 (14.2 MB/s) - ‘dynamodb_local_latest.tar.gz’ saved [44278951/44278951]

DynamoDBLocal_lib/
DynamoDBLocal.jar
DynamoDBLocal_lib/Apache-HttpComponents-HttpClient-4.5.x.jar
DynamoDBLocal_lib/Apache-HttpComponents-HttpCore-4.4.x.jar
DynamoDBLocal_lib/AwsFlowJava-1.0.jar
DynamoDBLocal_lib/AwsJavaSdk-Core-2.0.jar
DynamoDBLocal_lib/AwsJavaSdk-Core-Annotations-2.0.jar
DynamoDBLocal_lib/AwsJavaSdk-Core-Auth-2.0.jar
DynamoDBLocal_lib/AwsJavaSdk-Core

In [2]:
# 啟動後暫停
!java -Djava.library.path=./DynamoDBLocal_lib -jar DynamoDBLocal.jar -sharedDb &

Initializing DynamoDB Local with the following configuration:
Port:	8000
InMemory:	false
DbPath:	null
SharedDb:	true
shouldDelayTransientStatuses:	false
CorsParams:	null



KeyboardInterrupt: ignored

In [3]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.0 MB/s eta 0:00:00


In [4]:
import boto3
from botocore.config import Config
from botocore.exceptions import NoCredentialsError

In [5]:
import secrets
import string

allowed_chars = string.ascii_letters + string.digits

fake_key = ''.join(secrets.choice(allowed_chars) for _ in range(40))
fake_key

'NUtd3GoiyGuSKzt6CvQWXH7j39D1xeNLtgsycN6O'

In [6]:
dynamodb = boto3.resource(
    'dynamodb',
    endpoint_url='http://localhost:8000',
    aws_access_key_id='FAKEFAKEFAKEKEFAKEFA',
    aws_secret_access_key=fake_key,
    region_name='us-west-2'
)

In [7]:
# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

# Print out some data about the table.
print(table.item_count)

0


In [8]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/LangChain')
with open('env_vars.txt', 'r') as file:
  for line in file:
    key, value = line.strip().split('=')
    os.environ[key] = value

!pip install langchain==0.0.247 openai

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

def local_bot(session_id):
  llm = ChatOpenAI()
  prompt = ChatPromptTemplate(
      messages=[
          SystemMessagePromptTemplate.from_template(
              "You are a nice chatbot having a conversation with a human."
          ),
          MessagesPlaceholder(variable_name="chat_history"),
          HumanMessagePromptTemplate.from_template("{question}")
      ]
  )
  message_history = DynamoDBChatMessageHistory(table_name="SessionTable", endpoint_url='http://localhost:8000', session_id=session_id)
  memory = ConversationBufferMemory(
      memory_key="chat_history", chat_memory=message_history, return_messages=True
  )
  bot = LLMChain(
      llm=llm,
      prompt=prompt,
      verbose=True,
      memory=memory
  )
  return bot

In [13]:
local_bot("nick").run({"question": "hi I'm Nick"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Nick

> Finished chain.


'Hello Nick! How can I assist you today?'

In [14]:
local_bot("oliver").run({"question": "hi I'm Oliver"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Oliver

> Finished chain.


'Hello Oliver! How can I assist you today?'

In [15]:
local_bot("nick").run({"question": "你知道我的名字嗎"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi I'm Nick
AI: Hello Nick! How can I assist you today?
Human: 你知道我的名字嗎

> Finished chain.


'是的，你剛剛告訴我你的名字是Nick。有什麼我可以幫助你的嗎？'

In [16]:
history = DynamoDBChatMessageHistory(table_name="SessionTable", endpoint_url='http://localhost:8000', session_id="nick")
history.messages

[HumanMessage(content="hi I'm Nick", additional_kwargs={}, example=False),
 AIMessage(content='Hello Nick! How can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='你知道我的名字嗎', additional_kwargs={}, example=False),
 AIMessage(content='是的，你剛剛告訴我你的名字是Nick。有什麼我可以幫助你的嗎？', additional_kwargs={}, example=False)]